# Lab. TRIL - Net Zero

Engenharia Computacional para a Emissão Zero no Setor de Óleo e Gás

## Introdução 

Esse é um programa que realiza um processo de leitura, processamento e análise de um conjunto de dados de um reservatório, para encontrar classes de injetividade apresentadas com análise de gráfico.

As funcões a serem desenvolvidas são:
* read_df (recebe inputs csv ou df)
* calc_j (de J para J normalizado)
* proc (KDE e Bayesian Blocks)
* class (encontrar os pontos e classes)
* graf_j (analise dos graficos )

Esse projeto é desenvolvido por:<br>

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1 - Preparação de Files 

Essas funções devem receber arquivos csv (em uma formatação pré-estabelecida) que serem lidos, checados e iniciados.<br>
Os arquivos precisam:
* ter colunas nomeadas (de preferência em ordem alfabética)
* não ter valores NAN

In [34]:
'''função de leitura do arquivo csv para df'''

def read_df (arq):
    #leitura pelo pandas
    df = arq
        
    #limpeza (valores NAN)
    for i in df:
       if df[i].isna():
           print("df com valores NAN")
           df = df[i].fillna(0)
    #ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
    
    return df

In [ ]:
'''função de checagem de keys em dataframe '''

def checking(df):

    #recebe função de leitura
    df = read_df(df)

    #verificando se falta alguma coluna
    lista_base = ['RQI','pressao','pressao_inversa','distancia','permeabilidade','porosidade']
        
    for i in df.keys(): 
        if lista_base.count(i) == 0:            
            raise KeyError(msg_ERRO = f'variavel {i} não identificada')

    return(True,df)

## 2 - Calculo dos J's

In [ ]:
def normaliza_J (df, *J:list):
    
    #recebendo função de checagem
    df = checking(df)

    #atribuição 
    for j in J:
        
        if J == 'J1':
            J1 = df['RQI']
            df['J1'] = J1
                    
        if J == 'J2':
            J2 = df['RQI']*df['pressao']
            df['J2'] = J2

        if J == 'J3':
            J3 = df['RQI']*df['pressao_inversa']
            df['J3'] = J3
                            
        if J == 'J4':
            J4 = df['RQI'] * df['pressao'] * np.log(df['distancia']) 
            df['J4'] = J4

        if J == 'J5':
            J5 = df['RQI'] * df['pressao_inversa'] * np.log(df['distancia'])
            df['J5'] = J5
        
        if J == 'J6':
            J6 = df['permeabilidade'] * df['porosidade'] * np.log(df['distancia'])
            df['J6'] = J6
        
    #normalização
    for coluna in df.columns[6:]:
        df[f'{coluna}_normalizado'] = (df[coluna] - min(df[coluna])) / (max(df[coluna]) - min(df[coluna]))
    
    print(df.head())
    return df

## 3- Processamento Binning <br> (Bayesian Blocks ou KDE)

In [ ]:
def proc_binning(df, *J, binning): #recebe o df e o tipo de binning 

    if binning == 'kde':
        
        #bibliotecas
        from numpy import array, linspace
        from sklearn.neighbors import KernelDensity
        from scipy.misc import electrocardiogram
        from scipy.signal import argrelmin, find_peaks
        from sklearn.model_selection import GridSearchCV, LeaveOneOut
        import scipy.integrate as integrate

        #recebendo J normalizados
        df = normaliza_J(df, *J)

        #encontrar as curvas com band ideal
        #for coluna in J:
            #X = df[f'{coluna}_normalizado'].values[::].reshape(-1, 1) #retorna os valores do df com novo formato array
            #bandwidths = 10 ** np.linspace(0.01, 0.05) #são as medidas de bandwidth que escolhemos para testar
            #grid = GridSearchCV(KernelDensity(), {'bandwidth': bandwidths},cv=LeaveOneOut())
            #grid.fit(X1)

        min_i = 0
        max_i1 = df[f'{coluna}_normalizado'].shape[0]#pega a dimensao da coluna J1
        aux1 = max_i1 - min_i


        kde_J1= KernelDensity(bandwidth=0.01).fit(X) #por default: gaussian, bandwidth: 1.0
        s1 = np.linspace(0, 1.0, aux1) #retorna os pontos em uma distancia equidistante
        log1 = kde_J1.score_samples(s1.reshape(-1,1)) #calcula a probabilidade logarítmica de cada amostra sob o modelo
                    
        #calculo da integral
        integrate.trapezoid(np.exp(log1),s1)

        #encontrar picos
        
        #encontrar vales
        for coluna in J:
            X = sorted(df[f'{coluna}_normalizado'])
        
        

        


            


    elif binning == 'bb':

        from astropy.stats import bayesian_blocks

    return classes

In [ ]:
#teste de integral
#calculo da integral sobre o score_samples
# import scipy.integrate as integrate

## 4 - Pickle

In [ ]:
def pickle (df, fileout):
    import pickle as pkl


### Graficos dos Binnings

In [ ]:
def graf_binning(df, bins, *J, binning): #receberá o df, a quantidade de bins
                                         #os J e o tipo de binning
    
    if binning == 'kde':
    elif binning == 'bb':

## 5 - Análise dos Gráficos

## 6 - Menu

Aqui vamos chamar o pipeline e apresentar opções ao usuário chamando as funções

In [ ]:
print(' TRIL - NetZero '.center(30,'*'))
print(' Bem Vindo '.center(30,'*'))

In [15]:
#arq = str(print("Insira o caminho para o arquivo: ")) #recebemos o csv

arq = pd.read_csv("./dados/variaveis.csv", index=False)

In [35]:
#chamando a funçao de preparação
a = read_df(arq)
a

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### opções

In [ ]:
#o df precisa passar pelo calculo dos J's para prosseguir
#o usuário deve ter a opção de escolher quais calculos fazer? sim, quais J' normalizados

In [ ]:
opcao = 0
while opcao != 3:
    print('''   Qual tipo de processamento voce deseja executar?
            digite [1] para Bayesian Blocks
            digite [2] para KDE
            digite [3] para ambos
            digite [4] para voltar ao passo anterior ''')
    opcao = int(input())

    #if opcao == 1:
        # função BB
        #print("Processo de Binning por Bayesian Block completado")
        #print(imagens dos graficos de bayesian blocks para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 2:
        #função KDE
        #print("Processo de Binning por KDE completado")
        #print(imagens dos graficos de KDE para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 3:
        # função BB
        #função KDE
    
    #elif opcao == 4:
        #break #volta do começo

    #else:
        #print("Esse e um caractere invalido. Deseja recomeçar? Digite [0]")
        #opcao = 0

In [ ]:
print("Digite se deseja realizar a classificacao: s/n")
resposta = input()

#if resposta == 's':
    #retornar funcao de classificacao

#if resposta == 'n':
    #print("Fim do Programa")
else: 
    

In [ ]:
bandwidth = 0.01